## **LLM detector**

In [2]:
import pandas as pd 
import os 
print(os.getcwd())
os.chdir('../..')
print(os.getcwd())

/home/onyxia/ml-for-nlp/analysis/notebooks
/home/onyxia/ml-for-nlp


In [3]:
from src.libs.preprocess_dataframe_train_test import load_all_datasets

In [4]:
data = load_all_datasets()

### Global X, y
X_train = data["X_train"]
y_train = data["y_train"]
X_test = data["X_test"]
y_test = data["y_test"]

### Global dataframes
df_train = data["df_train"]
df_test = data["df_test"]

### By type: poetry, essay, story (train)
df_train_poetry = data["df_train_poetry"]
df_train_essay = data["df_train_essay"]
df_train_story = data["df_train_story"]
X_train_poetry = data["X_train_poetry"]
y_train_poetry = data["y_train_poetry"]
X_train_essay = data["X_train_essay"]
y_train_essay = data["y_train_essay"]
X_train_story = data["X_train_story"]
y_train_story = data["y_train_story"]

### By type: poetry, essay, story (test)
df_test_poetry = data["df_test_poetry"]
df_test_essay = data["df_test_essay"]
df_test_story = data["df_test_story"]
X_test_poetry = data["X_test_poetry"]
y_test_poetry = data["y_test_poetry"]
X_test_essay = data["X_test_essay"]
y_test_essay = data["y_test_essay"]
X_test_story = data["X_test_story"]
y_test_story = data["y_test_story"]


Time taken to load dataset: 0.009863138198852539 seconds
Time taken to load dataset: 0.007464408874511719 seconds
Time taken to load dataset: 0.28502821922302246 seconds
Time taken to load dataset: 0.011251688003540039 seconds
Time taken to load dataset: 0.006921529769897461 seconds
Time taken to load dataset: 0.07661628723144531 seconds
Time taken to load dataset: 0.011983394622802734 seconds
Time taken to load dataset: 0.009861946105957031 seconds
Time taken to load dataset: 0.011837244033813477 seconds


## **Roberta-based detector**

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "roberta-base-openai-detector"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)
model.eval() 


/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [5]:
def detect_score(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = softmax(logits, dim=1)
        return probs[0][1].item() 
    except Exception as e:
        print(f"Erreur avec le texte : {e}")
        return None


In [6]:
from tqdm import tqdm
tqdm.pandas()

df_train["llm_score"] = df_train["text"].progress_apply(detect_score)


100%|██████████| 1139/1139 [00:26<00:00, 42.37it/s]


In [7]:
df_train["llm_score"]

151    0.993539
710    0.774743
544    0.606294
263    0.000210
322    0.000177
         ...   
345    0.000225
171    0.999799
678    0.999805
594    0.553888
414    0.570419
Name: llm_score, Length: 1139, dtype: float64

## **Another part**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

def local_llm_score(text):
    prompt = build_prompt(text)
    output = generator(prompt, max_new_tokens=50, do_sample=True, temperature=0.7)[0]["generated_text"]
    
    import re
    match = re.search(r"([01](?:\.\d+)?)", output)
    if match:
        return float(match.group(1))
    return None


Fetching 10 files:   0%|          | 0/10 [03:37<?, ?it/s]


In [43]:
def build_prompt(text):
    return f"""<|system|>
You are an AI detector. You respond only with a float between 0.01 and 0.99 representing the probability the text is AI-generated.

<|user|>
{text}

What is the probability this was generated by an AI?
<|assistant|>
"""


In [44]:
local_llm_score("The weather is nice outside wanna go for a drink?")

KeyboardInterrupt: 

In [38]:
local_llm_score("Thank you for your respond, as an agent I would answer that there is no good answer to this question")

0.01

## **TRY**

In [5]:
from llama_cpp import Llama

llm = Llama(model_path="../llama_models/models/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf", 
    n_ctx=512,
    n_threads=8,           
    n_gpu_layers=32       
)


llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from ../llama_models/models/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   

llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:                      tokenizer.ggml.merges arr[str,61249]   = ["▁ t", "e r", "i n", "▁ a", "e n...
llama_model_loader: - kv  17:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  18:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  19:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  20:            tokenizer.ggml.padding_token_id u32              = 2
llama_model_loader: - kv  21:                    tokenizer.chat_template str              = {% for message in messages %}\n{% if m...
llama_model_loader: - kv  22:               general.quantization_version u32              = 2
l

In [ ]:
prompt = """<|system|>
You are a helpful assistant.

<|user|>
Can you explain how the moon affects the tides?

<|assistant|>
"""

response = llm(prompt, max_tokens=10)
print(response["choices"][0]["text"])
